# RunShell Input File Generator

In [46]:
%reload_ext autoreload
%autoreload 2


## create dir

In [47]:
import os
sim_name = 'gamma4'

cwd        = os.getcwd()
output_dir = os.path.join(cwd, 'NewSims', sim_name)
os.makedirs(output_dir, exist_ok=True)

# print(os.getcwd())
# print("Contents of", os.path.dirname(output_dir), "→", os.listdir(os.path.dirname(output_dir)))
# print("Contents of", output_dir, "→", os.listdir(output_dir))

## create all input files

In [48]:
from generate_inputs import *

create_input_files(
    sim_name     = sim_name,
    output_dir   = output_dir,
    domain_type  = 'rectangle',
    domain_params= (-10, 10, -1, 1),
    max_area     = 0.002,
    tols = (0,0)
)


## Initial configuration

In [54]:

from generate_inputs import plot_uv_surface, func_to_matlab_str
x = lambda u,v: u
y = lambda u,v: v
z = lambda u,v: 0 #np.sin(u)*np.cos(v)
umin = 2
umax = 4
vmin = 0
vmax = np.pi/3
# plot_uv_surface(x, y, z, umin, umax, vmin, vmax)

xpos = func_to_matlab_str(x)  
ypos = func_to_matlab_str(y)
zpos = func_to_matlab_str(z) 
print(xpos)
print(ypos)
print(zpos)


u
v
0


## GammaBar

In [55]:
import sympy as sp
from generate_inputs import calc_Gamma

u = sp.Symbol('u')
v = sp.Symbol('v')

#  example metric 
a = sp.Matrix([[1, 0], [0, u**2]])
Gamma_str = calc_Gamma(a, u, v)

print(Gamma_str)

[[['0', '0'], ['0', '-u']], [['0', '1./u'], ['1./u', '0']]]


## Params

In [56]:
thickness = 0.1
YoungsModulus = 10
PoisonRatio = 0.3
# "Lamme parameters" for connection energy
lambdaG = 100
muG = 100


## Generate RunShell input file

In [ ]:
from generate_inputs import make_in_file
output_dir = os.path.join(cwd, 'NewSims', sim_name)
params = {
    'vertices_file': os.path.relpath(os.path.join(output_dir, f"{sim_name}_Vertices"), os.getcwd()),
    'faces_file':  os.path.relpath(os.path.join(output_dir, f"{sim_name}_Faces"), os.getcwd()),
    'loops':50000,
    'save_every':400,
    'abar':['1','0','0','1'],
    'bbar':['0','0','0','0'],
    'thickness':str(thickness),
    'E':str(YoungsModulus),
    'nu':str(PoisonRatio),
    'pos0':[xpos,ypos,zpos],
    'lambdaG':str(lambdaG),
    'muG':str(muG),
    'gamma':Gamma_str,
    'thickness_adjust':'100',
    'metric_adjust':'1',
    'restart':'1'
}
in_file = make_in_file(params, output_dir, sim_name)



# Run

In [1]:
in_file_path = os.path.relpath(in_file, os.getcwd())
!./RunShell < {in_file_path}


NameError: name 'os' is not defined